In [1]:
from fastai.text import *

In [4]:
itos = pickle.load(open('itos_tfm.pkl', 'rb'))
vocab_sz = len(itos); vocab_sz

40478

In [5]:
model = get_transformer_lm(vocab_sz, 512, 12, 12, 768, 64, 768*4, 0.1, 0.1, 0.1, 0., act=Activation.GeLU, double_drop=False,
                        out_bias=False).cuda()

In [6]:
model.load_state_dict(torch.load('tfmer.pth'))

In [7]:
model = model.cuda()

In [8]:
stoi = {s:i for i,s in itos.items()}

Careful: words have a `</w>` flag in the vocabulary.

In [11]:
stoi['vanilla</w>']

'vanilla</w>'

In [10]:
itos[15000]

'vanilla</w>'

In [12]:
def textify(ids):       return ' '.join([itos[i].replace('</w>', '') for i in ids])
def numericalize(text): return [stoi[f'{w}</w>'] for w in text.split(' ')]

In [13]:
def predict(text, n_words, topk=10, temperature=1.):
    ids = numericalize(text)
    x = LongTensor(ids)[None].cuda()
    model.reset()
    model.eval()
    new_idx = []
    for k in range(n_words):
        out = F.softmax(model(x)[0][0,-1], dim=-1)
        if temperature != 1.: out.pow_(1 / temperature)
        values, indices = out.topk(topk)
        next_idx = indices.gather(-1, torch.multinomial(values, 1)).item()
        new_idx.append(next_idx)
        x =  LongTensor(ids + new_idx)[None].cuda()
        model.reset()
    return text + ' ' + textify(new_idx)

In [14]:
predict("this state has a population of", 50)

'this state has a population of more than fifty million . " \n " that \'s a pretty big number if it \'s just us , is n\'t it ? " \n " yes , and it \'s a lot more than us . " he looked at me , and his eyes were kind ,'